# Setup Environment

In [ ]:
%env UID=

env: UID=6530f9263773c5f7858b6b33


In [ ]:
!apt update && apt upgrade && apt-get install -y software-properties-common redis-server
!service redis-server start

!pip install https://github.com/robotics-4-all/commlib-py/archive/devel.zip -Uq
!pip install https://github.com/robotics-4-all/goalee/archive/devel.zip -Uq

!git clone https://github.com/robotics-4-all/streamsim.git
!cd streamsim/ && pip3 install -r requirements.txt
!cd streamsim && pip3 install .

!bash -c '[ ! -z appmaker-executor ] && git clone https://github.com/robotics-4-all/appmaker-executor.git'
!bash -c '[ ! -z goal-dsl ] && git clone https://github.com/robotics-4-all/goal-dsl.git && cd goal-dsl && git checkout devel && pip install .'
!bash -c '[ ! -z goaldsl-executor ] && git clone https://github.com/robotics-4-all/goaldsl-executor.git && cd goaldsl-executor && pip install -r requirements.txt'

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,789 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,235 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-s

# Start Executors

In [3]:
# Setup and Deploy GoalDSL-Executor
%env USE_REDIS=True
%env LOCAL_REDIS=True
%env BROKER_TYPE=MQTT
%env BROKER_HOST=locsys.issel.ee.auth.gr
%env BROKER_PORT=8883
%env BROKER_SSL=True
%env BROKER_USERNAME=sensors
%env BROKER_PASSWORD=issel.sensors
%env HEARTBEATS=False
%env DEBUG=False
%env WAIT_FOR_EXECUTION_TERMINATION=False

# Setup and Deploy StreamSim
!bash -c '[ -f streamsim.pid ] && pid=$(cat streamsim.pid) && kill $pid'
!nohup python3 streamsim/stream_simulator/bin/main.py ${UID} > streamsim.log 2>&1 & echo $! > streamsim.pid

# Setup and Deploy AppMaker-Executor
!bash -c '[ -f appexecutor.pid ] && pid=$(cat appexecutor.pid) && kill $pid'
!nohup python appmaker-executor/appmakerexecutor/src/appmakerexecutor/appmaker.py ${UID} > appexecutor.log 2>&1 & echo $! > appexecutor.pid

# Setup and Deploy GoalDSL-Executor
!bash -c '[ -f goalexecutor.pid ] && pid=$(cat goalexecutor.pid) && kill $pid'
!nohup python goaldsl-executor/executor.py > goalexecutor.log 2>&1 & echo $! > goalexecutor.pid

print("READY !!!")
# Check for errors
!tail -f goalexecutor.log appexecutor.log streamsim.log | grep -E "error|warning|Error|Warning|ERROR|WARNING|Reconnecting in"

env: USE_REDIS=True
env: LOCAL_REDIS=True
env: BROKER_TYPE=MQTT
env: BROKER_HOST=locsys.issel.ee.auth.gr
env: BROKER_PORT=8883
env: BROKER_SSL=True
env: BROKER_USERNAME=sensors
env: BROKER_PASSWORD=issel.sensors
env: HEARTBEATS=False
env: DEBUG=False
env: WAIT_FOR_EXECUTION_TERMINATION=False
READY !!!
           INFO     WARNING  Transport already connected - Skipping                   executor.py:23
           INFO     WARNING  Transport already connected - Skipping                   executor.py:23
           INFO     ERROR    [Scenario:EESTEC_Scenario] Fatal Goal <RobotCrash>       executor.py:23
ERROR : stream_simulator.robot : streamsim.67e3b511d4b2da5187c492a5.gn_robot: Crashed on a Wall
ERROR : stream_simulator.robot : streamsim.67e3b511d4b2da5187c492a5.gn_robot: Crashed on a Wall
ERROR : stream_simulator.robot : streamsim.67e3b511d4b2da5187c492a5.gn_robot: Crashed on a Wall
ERROR : stream_simulator.robot : streamsim.67e3b511d4b2da5187c492a5.gn_robot: Crashed on a Wall
ERROR : st

# Kill All

In [5]:
!bash -c '[ -f streamsim.pid ] && pid=$(cat streamsim.pid) && kill $pid'
!bash -c '[ -f appexecutor.pid ] && pid=$(cat appexecutor.pid) && kill $pid'
!bash -c '[ -f goalexecutor.pid ] && pid=$(cat goalexecutor.pid) && kill $pid'

bash: line 1: kill: (2570) - No such process
bash: line 1: kill: (2577) - No such process
bash: line 1: kill: (2595) - No such process
